In [1]:
import cv2
import numpy as np
import yaml
from yaml.loader import SafeLoader


In [2]:
# Load YAML
with open('data.yaml', mode='r') as f:
    data_yaml = yaml.load(f, Loader=SafeLoader)
labels = data_yaml['names']
print(labels)



['Car', 'Vacant', 'parking_spot']


In [3]:
# Load YOLO model
yolo = cv2.dnn.readNetFromONNX('./Model/weights/best.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# Load the image
img = cv2.imread('./Image_test_02.jpeg')
image = img.copy()
row, col, d = image.shape

In [4]:
# Convert image into square image (array)
max_rc = max(row, col)
input_image = np.zeros((max_rc, max_rc, 3), dtype=np.uint8)
input_image[0:row, 0:col] = image

# Resize the input image to the expected input size for YOLO
INPUT_WH_YOLO = 640  # Use the appropriate input size for your model
input_image_resized = cv2.resize(input_image, (INPUT_WH_YOLO, INPUT_WH_YOLO))

# Create blob
blob = cv2.dnn.blobFromImage(input_image_resized, 1 / 255.0, (INPUT_WH_YOLO, INPUT_WH_YOLO), swapRB=True, crop=False)


In [5]:
# Set input and perform prediction
yolo.setInput(blob)
preds = yolo.forward()  # Perform detection or prediction from YOLO

# Non-Maximum Suppression
# Step-1: Filter detection based on confidence (0.4) and probability score (0.25)
detections = preds[0]
boxes = []
confidences = []
classes = []

# Width and height of the image (input_image)
image_w, image_h = input_image.shape[:2]
x_factor = image_w / INPUT_WH_YOLO
y_factor = image_h / INPUT_WH_YOLO

for i in range(len(detections)):
    row = detections[i]
    confidence = row[4]  # Confidence of detection an object
    if confidence > 0.4:
        class_score = row[5:].max()  # Maximum probability from the classes
        class_id = row[5:].argmax()  # Get the index position at which max probability occurs

        if class_score > 0.25:
            cx, cy, w, h = row[0:4]
            # Construct bounding from four values
            # Left, top, width, and height
            left = int((cx - 0.5 * w) * x_factor)
            top = int((cy - 0.5 * h) * y_factor)
            width = int(w * x_factor)
            height = int(h * y_factor)

            box = np.array([left, top, width, height])

            # Append values into the list
            confidences.append(confidence)
            boxes.append(box)
            classes.append(class_id)


In [6]:
# Clean
boxes_np = np.array(boxes).tolist()
confidences_np = np.array(confidences).tolist()

# NMS
index = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.25, 0.45).flatten()

# Draw the Bounding Boxes
for ind in index:
    # Extract bounding box
    x, y, w, h = boxes_np[ind]
    bb_conf = int(confidences_np[ind] * 100)
    classes_id = classes[ind]
    class_name = labels[classes_id]

    text = f'{class_name}: {bb_conf}%'

    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.rectangle(image, (x, y - 30), (x + w, y), (255, 255, 255), -1)

    cv2.putText(image, text, (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 0), 1)




In [7]:
# Display results
cv2.imshow('Original', img)
cv2.imshow('YOLO Prediction', image)
cv2.waitKey(0)
cv2.destroyAllWindows()